In [26]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [27]:
#country_code読込
df_list = pd.read_csv("/home/yusuke_akada/notebook/Temp/county_code.csv")
df_list.head()

,code,country_name_J,country_name_E
0,IS,アイスランド,Iceland
1,IE,アイルランド,Ireland
2,AZ,アゼルバイジャン,Azerbaijan
3,AF,アフガニスタン,Afghanistan
4,US,アメリカ合衆国,United States


In [3]:
print(df_list.dtypes)

code              object
country_name_J    object
country_name_E    object
dtype: object


In [5]:
table_name = "dm_tmp.ya_country_code_20190920"
dtypes = ['string','string','string']
with ImpalaResource() as ir:
    ir.upload_dataframe(table_name, df_list, dtypes)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.102']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.102
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : 
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : root.adhoc_ai01_pool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'root.adhoc_ai01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.102]
INFO:impyla_service:New connection is opened.
INFO:impyla_se

In [33]:
%%slack_notify @yusuke_akada

for yymm in ['2019-07','2019-08']:
    query = (
        f"""
        with act as (
        select
        concat_ws('-',year,month) as yymm
        ,child_ssp_id
        ,country_code
        ,count(*)*100 as req
        ,count(smn_uid)*100 as sync_req
        ,sum(case when target_id > 0 then 1 else 0 end)*100 as res
        ,sum(case when bidden then 1 else 0 end)*100 as bidden
        ,sum(0) as imp
        ,sum(0) as click
        ,sum(0) as ctv
        ,sum(0) as vtv
        ,sum(0) as bid_price
        ,sum(0) as org_win_price
        ,sum(0) as win_price
        from dsplog_sampling.bid_response_time_one_percent_sampling bid
        where 
        concat_ws('-', year, month) = '{yymm}' 
        and (smn_uid IS NULL OR smn_uid NOT IN (1, 1111111111111, 2222222222222)) 
        and (test IS NULL OR test != TRUE) 
        and (advertiser_id IS NULL OR advertiser_id NOT IN (27,2014,4958,583))
        and ssp_id > 0
        and child_ssp_id > 0
        group by 1,2,3
        
        union all
        
        select
        concat_ws('-',year,month) as yymm
        ,child_ssp as childe_ssp_id
        ,country_code
        ,sum(0) as req
        ,sum(0) as sync_req
        ,sum(0) as res
        ,sum(0) as bidden
        ,sum(case when action_type in ('I') then 1 else 0 end) imp
        ,sum(case when action_type in ('C') then 1 else 0 end) click
        ,sum(0) ctv
        ,sum(0) vtv
        ,sum(case when action_type in ('I') then bidding_price else 0 end)/1000/1000/1000 bid_price
        ,sum(case when action_type in ('I') then original_winning_price else 0 end)/1000/1000 org_win_price
        ,sum(case when action_type in ('I') then winning_price else 0 end)/1000/1000 win_price
        from dsplog.adserver
        where
        concat_ws('-', year, month) = '{yymm}' 
        and action_type in ('I','C','CTV1','CTV2','VTV1','VTV2') and test is null
        and (smn_uid IS NULL OR smn_uid NOT IN (1, 1111111111111, 2222222222222)) 
        and (test IS NULL OR test != TRUE) 
        and (advertiser_id IS NULL OR advertiser_id NOT IN (27,2014,4958,583))
        and ssp_id > 0
        and child_ssp > 0
        group by
        1,2,3
        
        union all

        select
        concat_ws('-',year,month) as yymm
        ,child_ssp as childe_ssp_id
        ,country_code
        ,sum(0) as req
        ,sum(0) as sync_req
        ,sum(0) as res
        ,sum(0) as bidden
        ,sum(0) imp
        ,sum(0) click
        ,sum(case when action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        ,sum(0) bid_price
        ,sum(0) org_win_price
        ,sum(0) win_price
        from dsplog.adserver_conversion
        where
        concat_ws('-', year, month) = '{yymm}' 
        and action_type in ('I','C','CTV1','CTV2','VTV1','VTV2') and test is null
        and (smn_uid IS NULL OR smn_uid NOT IN (1, 1111111111111, 2222222222222)) 
        and (test IS NULL OR test != TRUE) 
        and (advertiser_id IS NULL OR advertiser_id NOT IN (27,2014,4958,583))
        and ssp_id > 0
        and child_ssp > 0
        group by
        1,2,3
            
        )
        select 
        yymm
        ,child_ssp_id
        ,country_code
        ,country_name_J
        ,sum(req) as req
        ,sum(sync_req) as sync_req
        ,sum(res) as res
        ,sum(bidden) as bidden
        ,sum(imp) as imp
        ,sum(click) as click
        ,sum(ctv) as ctv
        ,sum(vtv) as vtv
        ,sum(bid_price) as bid_price
        ,sum(org_win_price) as org_win_price
        ,sum(win_price) as win_price
        from act
        left join dm_tmp.ya_country_code_20190920 tmp
        on act.country_code = tmp.code        
        group by 1,2,3,4
        """
    )
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        df = df.astype({'org_win_price':float, 'win_price':float})
        gs = GSheet()
        url = "https://docs.google.com/spreadsheets/d/1IH2s_EdSUnkDsq_Bo9v00rQmZO82CWQKI0AcNq8JAW8/edit#gid=0"
        sheet_index = 0
        col_offset=0
        gs.append_data(url, sheet_index, df, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [22]:
print(df.dtypes)

yymmdd             object
child_ssp_id        int64
country_code       object
country_name_j     object
fd                 object
req                 int64
bidden              int64
res                 int64
imp                 int64
click               int64
ctv                 int64
vtv                 int64
bid_price         float64
org_win_price     float64
win_price         float64
dtype: object


country_code日本以外からのClick、Impをチェック

In [32]:
for yymm in ['2019-09']:
    query = (
        f"""
        with act as (      
        select
        concat_ws('-',year,month) as yymm
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,ads.strategy_id
        ,hie.strategy_name
        ,country_code
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(0) ctv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+cast(csm.margin as float)/100))/1000000 net_sales
        ,sum(winning_price*(1+cast(csm.margin as float)/100)/(1-cast(cam.margin as float)/100))/1000000 gross_sales
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.target_id = hie.target_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        where
        concat_ws('-', year, month) = '{yymm}' 
        and action_type in ('I','C','CTV1','CTV2','VTV1','VTV2') 
        and test is null
        and ads.country_code not in ('JP')
        and ads.child_ssp not in (18,22,29)
        group by 1,2,3,4,5,6
        having click > 0
        
        union all

        select
        concat_ws('-',year,month) as yymm
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,ads.strategy_id
        ,hie.strategy_name
        ,country_code
        ,sum(0) imp
        ,sum(0) click
        ,sum(case when action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(0) win_price
        ,sum(0) net_sales
        ,sum(0) gross_sales
        from dsplog.adserver_conversion ads
        inner join dm.hierarchies hie on ads.target_id = hie.target_id 
        where
        concat_ws('-', year, month) = '{yymm}' 
        and action_type in ('I','C','CTV1','CTV2','VTV1','VTV2') 
        and test is null
        and ads.country_code not in ('JP')
        and ads.child_ssp not in (18,22,29)
        group by 1,2,3,4,5,6
        having ctv > 0
            
        )
        select 
        yymm
        ,advertiser_id
        ,advertiser_name
        ,strategy_id
        ,strategy_name
        ,country_code
        ,country_name_J
        ,sum(imp) as imp
        ,sum(click) as click
        ,sum(ctv) as ctv
        ,sum(win_price) as win_price
        ,sum(net_sales) as net_sales
        ,sum(gross_sales) as gross_sales
        from act
        left join dm_tmp.ya_country_code_20190920 tmp
        on act.country_code = tmp.code 
        group by 1,2,3,4,5,6,7
        """
    )
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        df = df.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        url = "https://docs.google.com/spreadsheets/d/1IH2s_EdSUnkDsq_Bo9v00rQmZO82CWQKI0AcNq8JAW8/edit#gid=0"
        sheet_index = 1
        col_offset=0
        gs.append_data(url, sheet_index, df, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     